### 3-3．モデル作成
#### データの読み込みと確認

In [1]:
# Pandasの読み込み
import pandas as pd

# ファイルの読み込み
bank_df_new = pd.read_csv('bank-prep2.csv', sep=',')

# 先頭から5行目まで表示
bank_df_new.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,4Q,early,late,middle,long,short,one-more,zero,less,more
0,58,0,2143,1,0,5,261,1,-1,0,...,0,1,0,0,0,1,0,1,1,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,1,0,0,0,1,0,1,1,0


#### 不均衡データの均衡化

In [2]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.4.3
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: None
Author-email: None
License: MIT
Location: e:\anaconda3\envs\pbook_appx\lib\site-packages
Requires: scipy, numpy, scikit-learn
Required-by: 


In [3]:
!pip list

Package             Version   
------------------- ----------
absl-py             0.7.0     
astor               0.7.1     
backcall            0.1.0     
bleach              3.1.0     
certifi             2018.11.29
colorama            0.4.1     
cycler              0.10.0    
decorator           4.3.2     
entrypoints         0.3       
gast                0.2.2     
graphviz            0.10.1    
grpcio              1.16.1    
h5py                2.8.0     
imbalanced-learn    0.4.3     
ipykernel           5.1.0     
ipython             7.2.0     
ipython-genutils    0.2.0     
ipywidgets          7.4.2     
Janome              0.3.7     
jedi                0.13.2    
Jinja2              2.10      
jsonschema          2.6.0     
jupyter             1.0.0     
jupyter-client      5.2.4     
jupyter-console     6.0.0     
jupyter-core        4.4.0     
jupyterlab          0.35.3    
jupyterlab-server   0.2.0     
Keras               2.2.4     
Keras-Applications  1.0.6     
Keras-Pr

In [4]:
!pip install imbalanced-learn

In [5]:
# NumpyとRandomUnderSamplerの読み込み
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# データセットを説明変数と目的変数へ分割
X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new['y'])

# yが1と0のデータ件数をカウント
print(np.sum(Y == 1), np.sum(Y == 0))

# yが1のデータ件数に合わせてダウンサンプリング
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# yが1と0のデータ件数をカウント
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


**～～ここから追記～～**

In [6]:
from sklearn.feature_selection import SelectKBest

# 特徴量を5つ選択
selector = SelectKBest(k=5) 
selector.fit(X, Y)
mask = selector.get_support()

# どの変数を選択したかを確認
print(bank_df_new.drop('y', axis=1).columns)
print(mask)

Index(['age', 'default', 'balance', 'housing', 'loan', 'day', 'duration',
       'campaign', 'pdays', 'previous', 'divorced', 'married', 'single',
       'primary', 'secondary', 'tertiary', 'cellular', 'telephone', 'unknown',
       'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov',
       'oct', 'sep', 'non-worker', 'worker', '1Q', '2Q', '3Q', '4Q', 'early',
       'late', 'middle', 'long', 'short', 'one-more', 'zero', 'less', 'more'],
      dtype='object')
[False False False  True False False  True False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False  True  True False False False False]


In [7]:
# 選択した特徴量の列のみ取得
X_sel5 = selector.transform(X)
print(X_sel5)

[[  1  79   0   0   1]
 [  1  70   1   0   1]
 [  0 108   0   0   1]
 ...
 [  0 357   0   1   0]
 [  0 449   0   1   0]
 [  0 386   0   1   0]]


**～～ここまで追記～～**

#### 決定木モデルの作成と検証

In [8]:
# Scikit-learnの読み込み
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold交差検定
kf = KFold(n_splits=10, shuffle=True)
# モデル精度を格納する準備
scores = []
# データをシャッフルし、訓練データとテストデータに分割
for train_id, test_id in kf.split(X_sel5):
    # 訓練データを使ってモデルを作成
    x = X_sel5[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # テストデータにモデルを適用
    pred_y = clf.predict(X_sel5[test_id])
    # モデル精度を計算して格納
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# モデルの平均精度、標準偏差を確認
scores = np.array(scores)
print(scores.mean(), scores.std())

0.7182926829268291 0.008536585365853666


In [9]:
# Scikit-learnの読み込み
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 再現率、適合率の確認
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.675
0.72


#### モデルの精度向上

In [10]:
# モデル作成パラメータの確認
print(clf)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [11]:
# Scikit-learnの読み込み
from sklearn.model_selection import GridSearchCV

# パラメータ範囲設定
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# グリッドサーチの実行条件セット
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# グリッドサーチの実行
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['entropy'], 'max_depth': [2, 4, 6, 8, 10], 'min_samples_leaf': [10, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [12]:
# 最も高い精度とパラメータの組合せを表示
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.8085365853658537
{'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 20}
